In [ ]:
# interested in which 

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/crypto_data.csv')

In [5]:
df.columns = ['id', 'name', 'algorithm', 'is_trading', 'proof_type',
       'mined_coins', 'total_coins']

In [6]:
df.head()

,id,name,algorithm,is_trading,proof_type,mined_coins,total_coins
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [22]:
df.dtypes

id              object
name            object
algorithm       object
is_trading        bool
proof_type      object
mined_coins    float64
total_coins     object
dtype: object

In [24]:
df[df.total_coins.isna()]

,id,name,algorithm,is_trading,proof_type,mined_coins,total_coins


In [27]:
df.total_coins = df.total_coins.apply(lambda x: x.replace(" ", ""))
df.total_coins = df.total_coins.apply(lambda x: x.replace(".", ""))

In [28]:
df.total_coins = pd.to_numeric(df.total_coins)

In [29]:
df.dtypes

id              object
name            object
algorithm       object
is_trading        bool
proof_type      object
mined_coins    float64
total_coins      int64
dtype: object

In [30]:
# number of records
len(df)

1252

In [43]:
df.is_trading.value_counts()

True     685
False     59
Name: is_trading, dtype: int64

In [31]:
# distribution of algorithms
df.algorithm.value_counts()

Scrypt              424
X11                 197
SHA-256             131
X13                  57
PoS                  44
                   ... 
Stanford Folding      1
VBFT                  1
YescryptR16           1
IMesh                 1
TRC10                 1
Name: algorithm, Length: 95, dtype: int64

In [32]:
cnts = df.algorithm.value_counts()
cnts.loc[lambda x: x > 5]

Scrypt            424
X11               197
SHA-256           131
X13                57
PoS                44
CryptoNight        40
Quark              30
Equihash           26
NeoScrypt          21
NIST5              20
DPoS               19
Ethash             18
SHA-256D           18
Multiple           18
X15                14
Lyra2REv2           9
Blake               8
XEVAN               7
Lyra2Z              7
Lyra2RE             6
Skein               6
QuBit               6
CryptoNight-V7      6
PHI1612             6
Scrypt-n            6
Name: algorithm, dtype: int64

In [33]:
df.algorithm.isna().sum()

0

In [34]:
df.isna().sum()

id               0
name             0
algorithm        0
is_trading       0
proof_type       0
mined_coins    508
total_coins      0
dtype: int64

In [35]:
df.describe()

,mined_coins,total_coins
count,7.440000e+02,1.252000e+03
mean,3.883103e+09,1.686726e+15
std,3.867263e+10,4.012377e+16
min,-5.917978e+09,0.000000e+00
25%,6.322164e+05,1.800000e+07
50%,1.340427e+07,6.000000e+07
75%,9.399895e+07,4.000000e+08
max,9.899887e+11,1.004466e+18


In [ ]:
# total coins shows that some values are 0

In [38]:
df = df[~df.mined_coins.isna()]
df_clean = df[df.total_coins > 0]


In [39]:
df_clean.describe()

,mined_coins,total_coins
count,6.960000e+02,6.960000e+02
mean,3.777201e+09,1.581953e+15
std,3.958846e+10,3.822921e+16
min,-5.917978e+09,4.200000e+01
25%,6.583474e+05,2.100000e+07
50%,1.304427e+07,8.400000e+07
75%,8.358780e+07,5.000000e+08
max,9.899887e+11,1.004466e+18


In [41]:
df_clean.isna().sum()

id             0
name           0
algorithm      0
is_trading     0
proof_type     0
mined_coins    0
total_coins    0
dtype: int64

In [42]:
len(df_clean)

696

In [ ]:
# Evaluate the columns for enough values

In [45]:
df_clean.is_trading.value_counts()

True     637
False     59
Name: is_trading, dtype: int64

In [48]:
tmp = df_clean.algorithm.value_counts()
tmp

Scrypt                   229
X11                       98
SHA-256                   69
CryptoNight               24
Quark                     23
                        ... 
SkunkHash v2 Raptor        1
VeChainThor Authority      1
Ouroboros                  1
POS 2.0                    1
TRC10                      1
Name: algorithm, Length: 81, dtype: int64

In [49]:
tmp.loc[lambda x: x > 10]

Scrypt         229
X11             98
SHA-256         69
CryptoNight     24
Quark           23
Equihash        20
X13             20
PoS             18
NeoScrypt       17
Ethash          13
NIST5           11
Name: algorithm, dtype: int64

In [53]:
temp = df_clean.proof_type.value_counts()
temp

PoW                     326
PoW/PoS                 231
PoS                     106
DPoS                      8
PoC                       3
PoS/PoW                   2
DPoW                      1
LPoS                      1
DPoC                      1
Proof of Trust            1
POS / MN                  1
Pos                       1
DPOS                      1
Zero-Knowledge Proof      1
HPoW                      1
PoW + Hive                1
PoA                       1
POBh                      1
PoW and PoS               1
Proof of Authority        1
PoW/PoS                   1
TPoS                      1
PoW/PoW                   1
dPoW/PoW                  1
PoW/nPoS                  1
PoS/LPoS                  1
Name: proof_type, dtype: int64

In [54]:
temp.loc[lambda x: x > 5]

PoW        326
PoW/PoS    231
PoS        106
DPoS         8
Name: proof_type, dtype: int64